# 02 - Cálculo de Índices Espectrales
## Laboratorio 2: Detección de Cambios Urbanos - Chaitén

Este notebook documenta el cálculo de índices espectrales: NDVI, NDBI, NDWI y BSI.

## 1. Índices Espectrales Utilizados

| Índice | Fórmula | Interpretación |
|--------|---------|----------------|
| **NDVI** | (NIR - Rojo) / (NIR + Rojo) | Vegetación: valores altos (0.3-0.9) |
| **NDBI** | (SWIR - NIR) / (SWIR + NIR) | Urbano: valores positivos |
| **NDWI** | (Verde - NIR) / (Verde + NIR) | Agua: valores > 0.1 |
| **BSI** | ((SWIR+Rojo)-(NIR+Azul)) / ((SWIR+Rojo)+(NIR+Azul)) | Suelo desnudo |

In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from pathlib import Path

PROCESSED_DIR = Path('../data/processed')

## 2. Bandas Sentinel-2 Utilizadas

| Banda | Nombre | Resolución | Uso |
|-------|--------|------------|-----|
| B02 | Azul | 10m | BSI |
| B03 | Verde | 10m | NDWI |
| B04 | Rojo | 10m | NDVI, BSI |
| B08 | NIR | 10m | NDVI, NDBI, NDWI |
| B11 | SWIR1 | 20m | NDBI, BSI |

## 3. Implementación de Cálculo

```python
# Fórmulas implementadas en calculate_indices.py
eps = 1e-10  # Evitar división por cero

ndvi = (nir - rojo) / (nir + rojo + eps)
ndbi = (swir1 - nir) / (swir1 + nir + eps)
ndwi = (verde - nir) / (verde + nir + eps)
bsi = ((swir1 + rojo) - (nir + azul)) / ((swir1 + rojo) + (nir + azul) + eps)
```

## 4. Visualización de Índices

In [ ]:
def visualizar_indices(ruta_indices):
    """Visualiza los 4 índices de un archivo procesado."""
    with rasterio.open(ruta_indices) as src:
        ndvi = src.read(1)
        ndbi = src.read(2)
        ndwi = src.read(3)
        bsi = src.read(4)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    im1 = axes[0,0].imshow(ndvi, cmap='RdYlGn', vmin=-0.5, vmax=0.8)
    axes[0,0].set_title('NDVI (Vegetación)')
    plt.colorbar(im1, ax=axes[0,0])
    
    im2 = axes[0,1].imshow(ndbi, cmap='RdBu_r', vmin=-0.3, vmax=0.3)
    axes[0,1].set_title('NDBI (Urbano)')
    plt.colorbar(im2, ax=axes[0,1])
    
    im3 = axes[1,0].imshow(ndwi, cmap='Blues', vmin=-0.5, vmax=0.5)
    axes[1,0].set_title('NDWI (Agua)')
    plt.colorbar(im3, ax=axes[1,0])
    
    im4 = axes[1,1].imshow(bsi, cmap='YlOrBr', vmin=-0.3, vmax=0.3)
    axes[1,1].set_title('BSI (Suelo Desnudo)')
    plt.colorbar(im4, ax=axes[1,1])
    
    plt.tight_layout()
    return fig

In [ ]:
# Buscar archivos de índices procesados
archivos_indices = sorted(PROCESSED_DIR.glob('indices_*.tif'))

if archivos_indices:
    print(f"Archivos de índices encontrados: {len(archivos_indices)}")
    for f in archivos_indices:
        print(f"  - {f.name}")
    
    # Visualizar el primero
    print(f"\nVisualizando: {archivos_indices[0].name}")
    visualizar_indices(archivos_indices[0])
    plt.show()
else:
    print("No hay índices procesados. Ejecutar: python scripts/calculate_indices.py")

## 5. Interpretación de Valores

### NDVI (Normalized Difference Vegetation Index)
- **< 0.1**: Agua, nubes, nieve
- **0.1 - 0.2**: Suelo desnudo, rocas
- **0.2 - 0.4**: Vegetación dispersa, pastizales secos
- **0.4 - 0.6**: Vegetación moderada
- **> 0.6**: Vegetación densa, bosques

### NDBI (Normalized Difference Built-up Index)
- **< 0**: Vegetación, agua
- **0 - 0.2**: Mezcla suelo/urbano
- **> 0.2**: Áreas altamente urbanizadas

## 6. Ejecutar Procesamiento

```bash
python scripts/calculate_indices.py
```

Esto generará archivos `indices_YYYY_YYYYMMDD.tif` en `data/processed/`

---
**Anterior**: [01_descarga_datos.ipynb](01_descarga_datos.ipynb) | **Siguiente**: [03_deteccion_cambios.ipynb](03_deteccion_cambios.ipynb)